# **Name : Vinayak Murlidhar Rhatankar**

# **B.Tech AIML**

## ***Experiment No : 1. Simple ANN Implementation***

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
import matplotlib.pyplot as plt

(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

X_train = X_train / 255.0
X_test = X_test / 255.0

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, validation_split=0.2)

model.save("ANNModel.h5")

loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8636 - loss: 0.4839 - val_accuracy: 0.9538 - val_loss: 0.1636
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9600 - loss: 0.1393 - val_accuracy: 0.9658 - val_loss: 0.1141
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9741 - loss: 0.0915 - val_accuracy: 0.9681 - val_loss: 0.1006
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9808 - loss: 0.0665 - val_accuracy: 0.9741 - val_loss: 0.0892
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9857 - loss: 0.0490 - val_accuracy: 0.9702 - val_loss: 0.0987


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9679 - loss: 0.1046
Test Accuracy: 0.9723
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Sample Prediction: [7 2 1 0 4 1 4 9 6 9]
Actual Labels: [7 2 1 0 4 1 4 9 5 9]


# ***Flask Backend***

In [ ]:
from flask import Flask, render_template, request, jsonify
import numpy as np
import tensorflow as tf
from PIL import Image, ImageOps
import io
import os

model = tf.keras.models.load_model("./ANNModel.h5")

app = Flask(__name__)

def preprocess_image(image):
    """Convert uploaded image to MNIST format (28x28, grayscale, normalized)"""
    image = image.convert("L")  # Convert to grayscale
    image = ImageOps.invert(image)  # Invert colors (MNIST digits are white-on-black)
    image = image.resize((28, 28))  # Resize to 28x28 pixels
    image = np.array(image) / 255.0  # Normalize (0-1)
    image = image.reshape(1, 28, 28, 1)  # Add batch + channel dimensions
    return image

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({"error": "No file uploaded"})

    file = request.files['file']
    image = Image.open(io.BytesIO(file.read()))
    processed_image = preprocess_image(image)
    prediction = model.predict(processed_image)
    predicted_digit = int(np.argmax(prediction))
    confidence = float(np.max(prediction) * 100)

    return jsonify({"prediction": predicted_digit, "confidence": round(confidence, 2)})
#
# if __name__ == '__main__':
#     app.run(debug=True)

if __name__ == '__main__':
    port = int(os.environ.get("PORT", 5000))
    app.run(host="0.0.0.0", port=port)

# ***Frontend Code***

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MNIST Digit Classifier</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f9fafb;
            margin: 0;
            padding: 0;
        }

        header, footer {
            background-color: #3f51b5;
            color: white;
            padding: 15px;
            text-align: center;
        }

        footer {
            font-size: 14px;
        }

        .container {
            max-width: 450px;
            margin: 40px auto;
            background: #ffffff;
            padding: 25px;
            border-radius: 12px;
            box-shadow: 0 4px 15px rgba(0, 0, 0, 0.1);
            text-align: center;
        }

        .container h2 {
            margin-bottom: 15px;
            color: #333;
        }

        .upload-label {
            display: block;
            margin-bottom: 10px;
            font-weight: bold;
            color: #555;
        }

        input[type="file"] {
            padding: 10px;
            border: 2px solid #3f51b5;
            border-radius: 8px;
            background-color: #f0f2ff;
            cursor: pointer;
            width: 96%;
        }

        #imageContainer {
            margin-top: 20px;
            width: 100%;
            height: 220px;
            border: 1px solid #ddd;
            border-radius: 10px;
            display: flex;
            justify-content: center;
            align-items: center;
            background-color: white;
            overflow: hidden;
        }

        #imagePreview {
            max-width: 100%;
            max-height: 100%;
        }

        button {
            padding: 12px 20px;
            margin-top: 15px;
            background-color: #3f51b5;
            color: white;
            font-size: 16px;
            border: none;
            border-radius: 6px;
            cursor: pointer;
            width: 100%;
            transition: background-color 0.3s;
        }

        button:hover {
            background-color: #2e3b91;
        }

        #resultBox {
            margin-top: 20px;
            padding: 15px;
            background: #e8f5e9;
            border-radius: 8px;
            border-left: 5px solid #4caf50;
            display: none;
            color: #2e7d32;
            font-weight: bold;
        }

        @media (max-width: 500px) {
            .container {
                margin: 20px;
                padding: 15px;
            }
        }
    </style>
    <script>
        function previewImage(event) {
            let reader = new FileReader();
            reader.onload = function (e) {
                document.getElementById("imagePreview").src = e.target.result;
            };
            reader.readAsDataURL(event.target.files[0]);
        }

        function uploadImage() {
            let formData = new FormData();
            let file = document.getElementById("imageInput").files[0];
            if (!file) {
                alert("Please select an image first!");
                return;
            }
            formData.append("file", file);
            fetch("/predict", { method: "POST", body: formData })
                .then(response => response.json())
                .then(data => {
                    document.getElementById("resultBox").style.display = "block";
                    document.getElementById("resultBox").innerText =
                        "Predicted Digit: " + data.prediction;
                })
                .catch(err => console.error(err));
        }
    </script>
</head>
<body>

<header>
    <h1>MNIST Digit Classifier</h1>
</header>

<div class="container">
    <h2>Upload an Image </h2>
    <label class="upload-label" for="imageInput">Select a digit image (28x28 pixel preferred)</label>
    <input type="file" id="imageInput" accept="image/*" onchange="previewImage(event)">
    <div id="imageContainer">
        <img id="imagePreview" src="">
    </div>
    <button onclick="uploadImage()">Predict</button>
    <div id="resultBox"></div>
</div>

<footer>
    &copy; <span id="year"></span> MNIST Digit Classifier | Developed by <strong>Vinayak Rhatankar</strong>
</footer>

<script>
    document.getElementById("year").textContent = new Date().getFullYear();
</script>
</body>
</html>
